## 1. **Configure the Notebook:** 

In [14]:
# Import necessary libraries
import numpy as np
from xarray_healpy import HealpyGridInfo, HealpyRegridder
from pangeo_fish.grid import center_longitude
import xarray as xr
import pandas as pd
import xdggs
from pangeo_fish.hmm.estimator import EagerEstimator
from pangeo_fish.hmm.optimize import EagerBoundsSearch
from pangeo_fish.utils import temporal_resolution
from pangeo_fish.hmm.prediction import Gaussian1DHealpix,Gaussian2DCartesian
from tlz.functoolz import curry
from pint_xarray import unit_registry as ureg

# earth_radius defines the radius of the Earth used for distance calculations.
earth_radius = ureg.Quantity(6371, "km")
# maximum_speed sets the maximum allowable speed for the tagged fish.
maximum_speed = ureg.Quantity(60, "km / day")

# It will factor the allowed maximum displacement of the fish.
adjustment_factor = 5
# truncate sets the truncating factor for computed maximum allowed sigma for convolution process.
truncate = 4
tolerance = 1e-6
# Defines default chunk size for optimisation.
chunk_time = 24
default_chunk = {"time": chunk_time, "lat": -1, "lon": -1}
default_chunk_xy = {"time": chunk_time, "x": -1, "y": -1}
default_chunk_cells = {"time": chunk_time, "cells": -1}

In [15]:
import xarray as xr
emission = xr.open_dataset('../A19124/combined.zarr',
                           engine="zarr",
                           chunks={},
                           inline_array=True,
                           storage_options=None,
                        )
emission

<xarray.Dataset> Size: 409MB
Dimensions:     (y: 294, x: 659, time: 258)
Coordinates:
    cell_ids    (y, x) int64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    latitude    (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    longitude   (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: y, x
Data variables:
    final       (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    initial     (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    mask        (y, x) float64 2MB dask.array<chunksize=(294, 659), meta=np.ndarray>
    pdf         (time, x, y) float64 400MB dask.array<chunksize=(24, 659, 294), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0

In [40]:
def sigma_to_radian(sigma, earth_radius_km=6371):
    """
    Convert sigma (arc length) to radians.
    
    Parameters:
    - sigma: float, the arc length in the same unit as earth_radius_km.
    - earth_radius_km: float, the Earth's radius in kilometers (default: 6371 km).
    
    Returns:
    - float: sigma in radians.
    """
    return np.sqrt(4*np.pi/(12*4096**2))*sigma


# Apply the function to sigma = 1.06
sigma = 1.06292

sigma_radian = sigma_to_radian(sigma)

print(f"Sigma in radians: {sigma_radian}")
sigma_2_radian = sigma_to_radian(2)
print("sigma 2",sigma_2_radian)
earth_radius_ = xr.DataArray(earth_radius, dims=None)

timedelta = temporal_resolution(emission["time"]).pint.quantify().pint.to("h")
grid_resolution = earth_radius_ * emission["resolution"].pint.quantify()

maximum_speed_ = xr.DataArray(maximum_speed, dims=None).pint.to("km / h")
max_grid_displacement = maximum_speed_ * timedelta * adjustment_factor / grid_resolution
max_sigma = max_grid_displacement.pint.to("dimensionless").pint.magnitude / truncate

max_grid_displacement_radian = maximum_speed_ * timedelta * adjustment_factor / earth_radius_
max_sigma_radian = max_grid_displacement_radian.pint.to("dimensionless").pint.magnitude / truncate

# Applying max sigma over the emission matrices
emission.attrs["max_sigma"] = max_sigma_radian.item()
print(max_sigma.item())

print(max_sigma_radian)
emission

Sigma in radians: 0.00026555527939709026
sigma 2 0.0004996712441144963
1.963306279189256
0.0004905038455501491


<xarray.Dataset> Size: 409MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: cells
Data variables:
    final       (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    initial     (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    mask        (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    pdf         (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0
    max_sigma:  0.0004905038455501491

In [17]:
print(np.sqrt(4*np.pi/(12*4096**2)) * 1.06292)

0.00026555527939709026


In [18]:
emission = emission.stack(cells=("y", "x"))
emission = emission.drop_vars(['x','y'])
# Set required attributes cleanly
emission["cell_ids"].attrs["grid_name"] = "healpix"
#emission_1D["cell_ids"].attrs["nest"] = True
# Récupérer les attributs existants de cell_ids
attrs_to_keep = ["level", "grid_name"]
# should also have nest
emission["cell_ids"].attrs = {key: value for key, value in emission["cell_ids"].
                              attrs.items() if key in attrs_to_keep}
emission

/tmp/ipykernel_60585/299093097.py:2: DeprecationWarning: Deleting a single level of a MultiIndex is deprecated. Previously, this deleted all levels of a MultiIndex. Please also drop the following variables: {'cells'} to avoid an error in the future.
  emission = emission.drop_vars(['x','y'])


<xarray.Dataset> Size: 409MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: cells
Data variables:
    final       (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    initial     (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    mask        (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    pdf         (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0
    max_sigma:  0.0004905038455501491

In [19]:
emission_full_one = emission.copy()  # Crée une copie pour éviter de modifier directement les données
emission_full_one['pdf'] = xr.ones_like(emission_full_one['pdf'])

In [89]:
#emission_regridded_plus = emission_regridded_plus.compute()  # WHAT DOES IT CHANGE?
emission_1D = emission_full_one.assign_coords({'cell_ids':lambda ds:ds['cell_ids'].compute()}).sortby('cell_ids').pipe(xdggs.decode)
predictor_factory = curry(
    Gaussian1DHealpix,
    cell_ids=emission_1D["cell_ids"].data,
    grid_info=emission_1D.dggs.grid_info,
    truncate=5.0,
    weights_threshold=1e-8,
    pad_kwargs={"mode": "constant", "constant_value": 0},
    optimize_convolution=True,
)
emission_1D

<xarray.Dataset> Size: 409MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
  * cell_ids    (cells) int64 2MB 55996335 55996346 ... 58020672 58020674
    latitude    (cells) float64 2MB 42.45 42.46 42.47 ... 54.06 54.06 54.08
    longitude   (cells) float64 2MB -7.398 -7.378 -7.357 ... -1.149 -1.149
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: cells
Data variables:
    final       (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    initial     (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    mask        (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    pdf         (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=12, indexing_scheme=nested)
Attributes:
    grid_type:  healpix
    lat:        0
    level:      12
    lon:        0
    nside:      4096
    rot_lat:    0
    rot_lon:    0
    max_sigma:  0.0004905038455501491

In [90]:
##############################
# MANUELLEMENT AJOUT DU SIGMA
#############################
optimized = EagerEstimator(
            sigma=0.0004996712441144963,
            predictor_factory=predictor_factory)

optimized

EagerEstimator(predictor_factory=<class 'pangeo_fish.hmm.prediction.Gaussian1DHealpix'>, sigma=0.0004996712441144963, predictor=None)

In [91]:
emission_1D = emission_1D.compute()

In [92]:
emission_1D.initial.where(emission_1D.initial != 1, drop=True).dggs.explore()


Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [93]:
states_1D = optimized.predict_proba(emission_1D)
states_1D = states_1D.to_dataset().chunk(default_chunk_cells).persist()
states_1D

<xarray.Dataset> Size: 405MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
  * cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B 0.0002498
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
Dimensions without coordinates: cells
Data variables:
    states      (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>
Indexes:
    cell_ids  HealpixIndex(nside=12, indexing_scheme=nested)

In [95]:
states_1D.pipe(xdggs.decode).isel(time=-1).compute().states.dggs.explore(alpha=0.8)

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [85]:
import healpy as hp
# 
nside = int(emission_1D.attrs["nside"])  # nside 4096
npix = hp.nside2npix(nside)  # Nombre total  pixels HEALPix

full_map = np.zeros(npix)

cell_ids = emission_1D.cell_ids.data  # indices HEALPix des cells
values = emission_1D.initial.data  
full_map[cell_ids] = values 

# 3.  healpy
smoothed_map = hp.sphtfunc.smoothing(full_map, sigma=0.0004996712441144963,nest=True,lmax=nside,iter=60,use_pixel_weights=True)


smoothed_values = smoothed_map[cell_ids] 


In [86]:
import xarray as xr

# Create an xarray Dataset for the extracted smoothed values
extracted_dataset = xr.Dataset(
    {
        "smoothed": xr.DataArray(
            data=smoothed_map[emission_1D.cell_ids.values],  # Extract the relevant smoothed values
            dims=["cells"],
            coords={
                "cell_ids": emission_1D.cell_ids,
                "latitude": emission_1D.latitude,
                "longitude": emission_1D.longitude,
            },
        )
    },
    attrs=emission_1D.attrs,  # Copy any attributes from the original dataset
)

# Extract the 'smoothed' DataArray from the extracted dataset
smoothed_data_array = extracted_dataset["smoothed"]

# Now visualize the smoothed data using dggs.explore on the DataArray
smoothed_data_array.pipe(xdggs.decode).dggs.explore(alpha=0.8)

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…

In [17]:
import numpy as np
import healpy as hp
import xarray as xr
import matplotlib.pyplot as plt

# Paramètres HEALPix
nside = int(emission_1D.attrs["nside"])  # Récupérer la valeur nside
npix = hp.nside2npix(nside)  # Nombre total de pixels HEALPix

# Carte initiale
cell_ids = emission_1D.cell_ids.data  # Indices HEALPix des cellules
initial_values = emission_1D.initial.data  # États initiaux
full_map = np.zeros(npix)  # Carte pleine de zéros
full_map[cell_ids] = initial_values  # Remplir la carte avec les valeurs initiales

# Paramètres supplémentaires
fwhm = 0.00026555527939709026  # Paramètre de lissage (en radians)
mask = emission_1D.mask.notnull().data  # Masque des cellules valides

# Initialisation des états
alpha = full_map.copy()  # État initial

# Nombre de pas de temps
time_steps = 3

# Liste pour sauvegarder les états à chaque étape
alpha_time_steps = []

In [18]:
import healpy as hp
import numpy as np

# Paramètres
nside = int(emission_1D.attrs["nside"])  # Résolution HEALPix
npix = hp.nside2npix(nside)  # Nombre total de pixels HEALPix
fwhm = 0.00026555527939709026  # Largeur à mi-hauteur (en radians)

# Initialisation
alpha = emission_1D.initial.data  # Vos cellules initiales (taille 193746)
cell_ids = emission_1D.cell_ids.data  # Indices HEALPix de vos cellules pertinentes
time_steps = 3  # Nombre d'étapes de temps (réduisez pour les tests)
alpha_time_steps = []  # Pour enregistrer l'évolution temporelle

In [19]:

#for t in range(1, time_steps + 1):

full_alpha = np.zeros(npix)  # Carte complète
full_alpha[cell_ids] = alpha  # Remplir avec vos cellules initiales

smoothed_full_alpha = hp.sphtfunc.smoothing(full_alpha, fwhm=fwhm, nest=True)
print("size smoothing", smoothed_full_alpha)

alpha = smoothed_full_alpha[cell_ids]

observation_pdf = emission_1D.pdf.isel(time=0).data
alpha *= observation_pdf


total_sum = np.nansum(alpha)
alpha /= total_sum


alpha_time_steps.append(alpha)


size smoothing [ 1.03452394e-08 -9.66442316e-08 -6.36607250e-08 ...  3.59837997e-07
 -2.00506676e-07  2.67048007e-07]
Évolution de alpha sur 3 étapes calculée.


In [ ]:
len(alpha_time_steps[0])

In [ ]:
# Convertir les résultats en un xarray Dataset pour l'analyse
alpha_dataset = xr.Dataset(
    {
        "alpha": xr.DataArray(
            data=np.array(alpha_time_steps),
            dims=["time", "cells"],
            coords={
                "time": emission_1D.time[1:].data,  # Temps correspondants
                "cell_ids": emission_1D.cell_ids,
                "latitude": emission_1D.latitude,
                "longitude": emission_1D.longitude,
            },
        )
    },
    attrs=emission_1D.attrs,  # Copier les attributs originaux
)

In [ ]:
len(alpha_time_steps[0])

In [15]:
states_1D_from_2D = xr.open_dataset('states_from_emission_2D_emissionfullone.zarr',
                           engine="zarr",
                           chunks={},
                           inline_array=True,
                           storage_options=None,
                        )
states_1D_from_2D


<xarray.Dataset> Size: 408MB
Dimensions:     (cells: 193746, time: 258)
Coordinates:
    cell_ids    (cells) int64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    latitude    (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    longitude   (cells) float64 2MB dask.array<chunksize=(193746,), meta=np.ndarray>
    resolution  float64 8B ...
  * time        (time) datetime64[ns] 2kB 2022-06-13T12:00:00 ... 2022-06-24T...
    x           (cells) int64 2MB dask.array<chunksize=(48437,), meta=np.ndarray>
    y           (cells) int64 2MB dask.array<chunksize=(48437,), meta=np.ndarray>
Dimensions without coordinates: cells
Data variables:
    states      (time, cells) float64 400MB dask.array<chunksize=(24, 193746), meta=np.ndarray>

In [16]:
states_1D_from_2D.pipe(xdggs.decode).isel(time=0).compute().states.dggs.explore(alpha=0.8)

Map(custom_attribution='', layers=(SolidPolygonLayer(filled=True, get_fill_color=arro3.core.ChunkedArray<Fixed…